In [5]:
# !pip install transformers
# !pip install datasets==2.21.0
# !pip install wandb

In [9]:
import torch
from transformers import GPT2ForQuestionAnswering, GPT2TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from torch.nn.utils.rnn import pad_sequence
import os
import wandb
from transformers.integrations import WandbCallback
import numpy as np

from datasets import load_metric # used in compute_metrics
from transformers.trainer_utils import EvalPrediction
from typing import Dict, List


def freeze_layers(model, variant_type):
    if variant_type == "noNorm":
        for name, param in model.named_parameters():
            if "ln" in name:
                param.requires_grad = False
    elif variant_type == "AttnOnly":
        for name, param in model.named_parameters():
            if "ln_2" in name:  # Freeze FFN layer norm
                param.requires_grad = False
    elif variant_type == "FFOnly":
        for name, param in model.named_parameters():
            if "ln_1" in name:  # Freeze attention layer norm
                param.requires_grad = False
    # For baseModel, we don't freeze any layers

def prepare_squad_dataset(tokenizer):
    dataset = load_dataset("squad")

    def preprocess_function(examples):
        questions = [q.strip() for q in examples["question"]]
        contexts = [c.strip() for c in examples["context"]]

        # Tokenize questions and contexts together
        tokenized_examples = tokenizer(
            [tokenizer.cls_token + q for q in questions], # new addition
            contexts,
            max_length=384,
            truncation="only_second",
            stride=128,
            return_overflowing_tokens=True,
            padding="max_length",
            return_offsets_mapping=True,
        )

        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        offset_mapping = tokenized_examples["offset_mapping"]

        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = 0

            sequence_ids = tokenized_examples.sequence_ids(i)

            sample_index = sample_mapping[i]
            answers = examples["answers"][sample_index]

            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

    tokenized_datasets = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
    )

    return tokenized_datasets


def fine_tune_model(model, tokenizer, dataset, output_dir, variant, num_train_epochs=3):
    # Initialize wandb run
    wandb.init(project=f"GPT-Valkyrie_LN-124m__{variant}__SQuAD_Simple", reinit=True)
    run_name = wandb.run.name

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,  # Changed from max_steps to num_train_epochs
        per_device_train_batch_size=24,
        per_device_eval_batch_size=24,
        warmup_ratio=0.1,  # Changed from warmup_steps to warmup_ratio
        weight_decay=0.01,
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        report_to="wandb",
        run_name=run_name,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        callbacks=[WandbCallback()],
    )

    trainer.train()
    wandb.finish()
    return trainer.model, run_name

In [12]:
# MAIN LOOP
wandb.login()

variants = ["noNorm", "AttnOnly", "FFNonly", "baseModel"]

from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.cls_token = "[CLS]"
tokenizer.add_special_tokens({'cls_token': '[CLS]'})

dataset = prepare_squad_dataset(tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
for variant in variants:
    print(f"Processing {variant} model...")

    # Use the correct base model for each variant
    model_path = f"shng2025/GPT-Valkyrie_LN-124m__{variant}__"
    model = GPT2ForQuestionAnswering.from_pretrained(model_path)

    freeze_layers(model, variant)

    output_dir = f"./results/{variant}"
    fine_tuned_model, run_name = fine_tune_model(model, tokenizer, dataset, output_dir, variant)

    # Save the model locally
    local_save_dir = f"./local_models/GPT-Valkyrie_LN-124m__{variant}__SQuAD"
    fine_tuned_model.save_pretrained(local_save_dir)
    tokenizer.save_pretrained(local_save_dir)
    print(f"Model saved locally to {local_save_dir}")

    # Push the model to your HuggingFace Hub repository
    new_repo_name = f"shng2025/GPT-Valkyrie_LN-124m__{variant}__SQuAD"
    fine_tuned_model.push_to_hub(new_repo_name, branch=run_name)
    tokenizer.push_to_hub(new_repo_name, branch=run_name)
    print(f"Model pushed to HuggingFace Hub: {new_repo_name}, branch: {run_name}")

Processing noNorm model...


Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at shng2025/GPT-Valkyrie_LN-124m__noNorm__ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train/epoch,▁▁▃▃▄▄▆▆██
train/global_step,▁▁▃▃▅▅▆▆██
train/grad_norm,██▃▃▂▂▃▃▁▁
train/learning_rate,▁▁▃▃▅▅▆▆██
train/loss,██▆▆▂▂▂▂▁▁
train/epoch,0.00452
train/global_step,50
train/grad_norm,32.81399
train/learning_rate,5e-05
train/loss,4.9877


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
200,4.567500,4.353150


...

eval/loss,▁▁
eval/runtime,▁▁
eval/samples_per_second,▁▁
eval/steps_per_second,▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███████
train/grad_norm,██▅▅▄▄▅▅▃▃▃▃▆▆▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂
train/learning_rate,▂▂▄▄▅▅▇▇████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁
train/loss,██▆▆▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▂▂▂▂▂▂▁▁▁▁▂▂
eval/loss,3.86157
eval/runtime,124.0682


Model saved locally to ./local_models/GPT-Valkyrie_LN-124m__baseModel__SQuAD


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Model pushed to HuggingFace Hub: shng2025/GPT-Valkyrie_LN-124m__baseModel__SQuAD, branch: genial-sound-1


In [ ]:
for variant in variants:
    print(f"Processing {variant} model...")

    # Use the correct base model for each variant
    model_path = f"shng2025/GPT-Valkyrie_RMSN-124m__{variant}__"
    model = GPT2ForQuestionAnswering.from_pretrained(model_path)

    freeze_layers(model, variant)

    output_dir = f"./results/{variant}"
    fine_tuned_model, run_name = fine_tune_model(model, tokenizer, dataset, output_dir, variant)

    # Save the model locally
    local_save_dir = f"./local_models/GPT-Valkyrie_RMSN-124m__{variant}__SQuAD"
    fine_tuned_model.save_pretrained(local_save_dir)
    tokenizer.save_pretrained(local_save_dir)
    print(f"Model saved locally to {local_save_dir}")

    # Push the model to your HuggingFace Hub repository
    new_repo_name = f"shng2025/GPT-Valkyrie_RMSN-124m__{variant}__SQuAD"
    fine_tuned_model.push_to_hub(new_repo_name, branch=run_name)
    tokenizer.push_to_hub(new_repo_name, branch=run_name)
    print(f"Model pushed to HuggingFace Hub: {new_repo_name}, branch: {run_name}")